# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя? Обоснуйте выбор теста. Сделайте вывод по гипотезе.

In [58]:
# исходные данные
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

In [59]:
import pandas as pd

In [60]:
# так как исходные в списках разной длины, то можно использовать concat, чтобы объединить их в dataframe
lists = [petersburg, magadan]
df = pd.concat([pd.Series(x) for x in lists], axis=1)
df.columns = ['petersburg','magadan']
df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


In [61]:
#в столбцах есть пропуски. заполнил пропущенные значения медианными значениями, чтобы не исказить распределение
fill_df = df.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'petersburg': fill_df['petersburg'].median()
}
#заполняем пропуски в соответствии с заявленным словарем
fill_df = fill_df.fillna(values)
fill_df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


# Выбор теста на корреляцию

Чтобы выбрать тип корреляции руководствуемся следующим:
* Если данные распределены нормально, то используем корреляцию Пирсона
* Если данные распределены ненормально, то:
    а) есть выбросы и ошибки - используем корреляцию Кендалла;
    б) нет выбросов и ошибок - используем корреляцию Спирмена

In [62]:
#тестируем данные на нормальное распределение
#задаем исходные параметры
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
alpha = 0.05

# Тест Шапиро-Уилка (для проверки на нормальное распределение)

In [63]:
from scipy.stats import shapiro

In [64]:
_, p = shapiro(fill_df)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)
 
 
#тест Д`Агостино не используем, т.к. величина выборки слишком мала, чтобы тест показал корректные результаты

p=0.101
Данные распределены нормально


In [65]:
# данные распределены нормально, поэтому используем корреляцию Пирсона
round(fill_df.corr(method='pearson'),6)

#значение корреляции меньше - 0,3, т.е. связь слабая

,petersburg,magadan
petersburg,1.000000,0.256665
magadan,0.256665,1.000000


In [71]:
#можно ещё таким образом посчитать корреляцию Пирсона и заодно проверить наличие или отсутствие зависимости

from numpy.random import rand
from scipy.stats import pearsonr


corr, p = pearsonr(fill_df['petersburg'], fill_df['magadan'])

print(corr,p)

if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.25666515110501303 0.5394705530383969
0.5394705530383969 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.


## Вывод:
* Данные выборок распределены нормально. 
* Используется корреляция Пирсона. 
* Судя по значению корреляции (меньше 0,3) связь слабая, тем не менее мы не можем отвергнуть гипотезу от отсутствии зависимости между данными.

Отвечаем на вопрос: Какова разница в среднем размере мидии в зависимости от города-производителя?

В данном случае: зависимая переменная - одна и количественная (размер мидий), сравниваются две группы(из разных совокупностей), при этом в каждой из групп до 30 объектов, то используем t-тест

In [72]:
from scipy.stats import ttest_ind

H0 = 'Нет значимой разницы между размерами мидий в разных городах.'
Ha = 'Есть значимая разница между размерами мидий в разных городах.'

def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['petersburg'], df['magadan'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(fill_df)


*** Результаты независимого T-теста ***
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий в разных городах.


In [80]:
#Считаем различие в средних значениям мидий в зависимости от города произвдителя

pet = round(df['petersburg'].mean(),4)
mag = round(df['magadan'].mean(),4)

display(f"Среднее значение в Петербурге - {pet}")
display(f"Среднее значение в Магадане - {mag}")
display(f"Отличие между средними значениями - {round(pet-mag, 4)}")

'Среднее значение в Петербурге - 0.1034'

'Среднее значение в Магадане - 0.078'

'Отличие между средними значениями - 0.0254'

# Вывод:
* В качестве теста был выбран t-тест (зависимая переменная - одна и количественная (размер мидий), сравниваются две группы(из разных совокупностей), при этом в каждой из групп до 30 объектов).
* По результатам теста подтвердилось, что существует значимая разница между размерами мидий в разных городах
* Действительно: Среднее значение мидий в Петербурге - 0.1034, в Магадане - 0.078, отличие между средними значениями - 0.0254, т.е. в Петербурге в среднем мидии почти на треть больше.